# Задание
Задан взвешенный граф 𝐺 и константа 𝐿 = Число вершин G \ 10 с округлением в
меньшую сторону.
Задача. Требуется в графе 𝐺 найти такое остовное дерево минимального веса,
что число листьев не превосходит константы 𝐿 и степени всех вершин
ограничены 3 (бинарное дерево). 
***

# Решение
### Алгоритм
Модифицированный алгоритм Прима

### Вход в виде файла
Граф `G` можно задать в файле `.txt` следующим образом:  
  
`0 1 1`  
`0 2 3`  
где формат строки: `<вершина 1> <вершина 2> <вес ребра>`
 
В случае нескольких графов в 1 файле, разделитель - `пустая строка`  
Конец файла - `пустая строка`.

# Зависимости

In [16]:
import networkx as nx
import random
import math

# Чтение из файла

In [17]:
def read_graph_from_file(file_path):
    graphs = []
    G = nx.Graph()
    with open(file_path, 'r') as file:
        lines = file.readlines()
    for line in lines:
        if line == '\n':
            graphs.append(G)
            G = nx.Graph()
        u, v, w = map(int, line.split())
        G.add_edge(u, v, weight=w)
    return G

# Генерация случайного графа

### Генерация

In [18]:
def generate_random_graph(num_vertices, num_edges, weight_range):
    G = nx.gnm_random_graph(num_vertices, num_edges)
    for (u, v) in G.edges():
        G[u][v]['weight'] = random.randint(*weight_range)
    return G


# Запись в файл
Модификация for_website позволяет просто скопировать данные из файла и построить граф на сайте [Graph Online](https://graphonline.ru/create_graph_by_edge_list)

In [19]:
def write_graph_to_file(G, file_path, mode = 'a', for_website = False):
    with open(file_path, mode=mode) as file:
        for (u, v, data) in G.edges(data=True):
            file.write(f"{u}{'-' if for_website else ' '}{v}{' ' + data['weight'] if not for_website else ''}\n")
        file.write('\n')

# Код
### Описание алгоритма:  
В основе лежит алгоритм Прима для нахождения минимального остовного дерева (MST)  
### Модификация:  
* Количество листьев не должно превышать L 
* Степени всех вершин были ограничены 3.

In [22]:
def prim_mst(G):
    return nx.minimum_spanning_tree(G, algorithm='prim')

def modify_mst(G, L):
    for node in list(G.nodes):
        while G.degree(node) > 3:
            edges = list(G.edges(node, data=True))
            edges.sort(key=lambda x: x[2]['weight'], reverse=True)
            G.remove_edge(edges[0][0], edges[0][1])

    leaves = [node for node in G.nodes if G.degree(node) == 1]
    while len(leaves) > L:
        leaf = leaves.pop(0)
        neighbors = list(G.neighbors(leaf))
        if neighbors:
            G.remove_edge(leaf, neighbors[0])
        leaves = [node for node in G.nodes if G.degree(node) == 1]

    return G


def find_mst(G, for_website = False):
    L = math.floor((len(G.nodes) - 10) / 10)
    mst = prim_mst(G)
    modified_mst = modify_mst(mst, L)
    write_graph_to_file(modified_mst, 'output.txt', 'a', for_website=for_website)



# Тестирование

In [25]:
import time
graphs = []
for i in range (10):
    G = generate_random_graph(1000, 2000, (1, 100))
    graphs.append(G)
    
for graph in graphs:
    start_time = time.time()
    mst = find_mst(graph, for_website=True)
    end_time = time.time()
    print(f"Time: {end_time - start_time:.6f} seconds")

Time: 0.173970 seconds
Time: 0.150473 seconds
Time: 0.165120 seconds
Time: 0.145886 seconds
Time: 0.159298 seconds
Time: 0.154061 seconds
Time: 0.162601 seconds
Time: 0.174378 seconds
Time: 0.160818 seconds
Time: 0.159587 seconds


В выводе показано время работы алгоритма на графах с 1000 вершин и 2000 ребер.